In [3]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pandas as pd
pd.options.mode.chained_assignment = None

In [7]:
MergedSoFifaFbref = pd.read_csv('./data/MergedSoFifaFbref.csv',keep_default_na=False)
fbref_MergedMatchStats = pd.read_csv('./data/fbref_MergedMatchStats.csv',keep_default_na=False)

In [ ]:
fbref_MergedMatchStats.head(1)


,home_Ast,home_Blocks,home_Carries,home_Carries_PrgC,home_CrdR,home_CrdY,home_GCA,home_Gls,home_Int,home_PK,...,away_offsides,away_possession,away_score,away_tackles,away_team,away_throw_ins,away_total_players_stats,away_touches44,away_xAG,away_xG
0,0,4,295,21,1,1,1,1,4,0,...,1,51%,4,15,West Ham United,12,14,621,0.9,3.1


In [ ]:
fbref_MergedMatchStats.columns

Index(['home_Ast', 'home_Blocks', 'home_Carries', 'home_Carries_PrgC',
       'home_CrdR', 'home_CrdY', 'home_GCA', 'home_Gls', 'home_Int', 'home_PK',
       'home_PKatt', 'home_Passes_Att', 'home_Passes_Cmp',
       'home_Passes_CmpPercentage', 'home_Passes_PrgP', 'home_SCA', 'home_Sh',
       'home_SoT', 'home_Take_Ons_Att', 'home_Take_Ons_Succ', 'home_Tkl',
       'home_Touches21', 'home_aerials_won', 'home_captain', 'home_clearances',
       'home_corners', 'home_crosses', 'home_fbrefMatchId', 'home_formation',
       'home_fouls', 'home_goal_kicks', 'home_interceptions',
       'home_is_home_team', 'home_long_balls', 'home_manager', 'home_minutes',
       'home_npxG', 'home_offsides', 'home_possession', 'home_score',
       'home_tackles', 'home_team', 'home_throw_ins',
       'home_total_players_stats', 'home_touches44', 'home_xAG', 'home_xG',
       'away_Ast', 'away_Blocks', 'away_Carries', 'away_Carries_PrgC',
       'away_CrdR', 'away_CrdY', 'away_GCA', 'away_Gls', 'away_Int'

In [9]:
def MergeMatchStatsSoFifa(MergedSoFifaFbref,fbref_MergedMatchStats):
    columns_MergeMatchStatsSoFifa = fbref_MergedMatchStats.columns.values.tolist()
    columns_MergeMatchStatsSoFifa.extend(['home_attack_rating','home_mid_rating','home_defend_rating','away_attack_rating','away_mid_rating','away_defend_rating'])
    df_MergeMatchStatsSoFifa = pd.DataFrame(columns=columns_MergeMatchStatsSoFifa)
    attack_line = ["LS", "ST", "RS","LW", "LF", "CF", "RF", "RW"]
    mid_line = ["LAM", "CAM", "RAM","LM", "LCM", "CM", "RCM", "RM","LDM", "CDM", "RDM"]
    defend_line = ["LB", "LCB", "CB", "RCB", "RB","LWB","RWB"]

    for i in range(0,len(fbref_MergedMatchStats)):
    # for i in range(0,10):
        matchStat = fbref_MergedMatchStats.iloc[i]
        matchStat_row = fbref_MergedMatchStats.loc[i:i]
        home_squad = MergedSoFifaFbref.loc[(MergedSoFifaFbref['fbrefMatchId']==matchStat['home_fbrefMatchId']) & (MergedSoFifaFbref['team']==matchStat['home_team'])]
        away_squad = MergedSoFifaFbref.loc[(MergedSoFifaFbref['fbrefMatchId']==matchStat['away_fbrefMatchId']) & (MergedSoFifaFbref['team']==matchStat['away_team'])]
        home_attack_rating = []
        home_mid_rating = []
        home_defend_rating = []
        away_attack_rating = []
        away_mid_rating = []
        away_defend_rating = []
        for i in range (0,len(home_squad)):
            player = home_squad.iloc[i]
            all_positions = player['all_positions']
            position = all_positions.split(' ')[0]
            if position in attack_line:
                home_attack_rating.append(player['overall_rating'])
            elif position in mid_line:
                home_mid_rating.append(player['overall_rating'])
            elif position in defend_line:
                home_defend_rating.append(player['overall_rating'])
        
        for i in range(0,len(away_squad)):
            player = away_squad.iloc[i]
            all_positions = player['all_positions']
            position = all_positions.split(' ')[0]
            if position in attack_line:
                away_attack_rating.append(player['overall_rating'])
            elif position in mid_line:
                away_mid_rating.append(player['overall_rating'])
            elif position in defend_line:
                away_defend_rating.append(player['overall_rating'])
        home_attack_rating = list(map(int,home_attack_rating))
        home_mid_rating = list(map(int,home_mid_rating))
        home_defend_rating = list(map(int,home_defend_rating))
        away_attack_rating = list(map(int,away_attack_rating))
        away_mid_rating = list(map(int,away_mid_rating))
        away_defend_rating = list(map(int,away_defend_rating))

        if (len(home_attack_rating) == 0):
            home_attack_rating = 0
        else: home_attack_rating = sum(home_attack_rating)/len(home_attack_rating)
        if (len(home_mid_rating) == 0):
            home_mid_rating = 0
        else: home_mid_rating = sum(home_mid_rating)/len(home_mid_rating)
        if (len(home_defend_rating) == 0):
            home_defend_rating = 0
        else: home_defend_rating = sum(home_defend_rating)/len(home_defend_rating)
        if (len(away_attack_rating) == 0):
            away_attack_rating = 0
        else: away_attack_rating = sum(away_attack_rating)/len(away_attack_rating)
        if (len(away_mid_rating) == 0):
            away_mid_rating = 0
        else: away_mid_rating = sum(away_mid_rating)/len(away_mid_rating)
        if (len(away_defend_rating) == 0):
            away_defend_rating = 0
        else: away_defend_rating = sum(away_defend_rating)/len(away_defend_rating)
        
        matchStat_row.loc[:,'home_attack_rating'] = round(home_attack_rating,2) 
        matchStat_row.loc[:,'home_mid_rating'] = round(home_mid_rating,2) 
        matchStat_row.loc[:,'home_defend_rating'] = round(home_defend_rating,2)
        matchStat_row.loc[:,'away_attack_rating'] = round(away_attack_rating,2)
        matchStat_row.loc[:,'away_mid_rating'] = round(away_mid_rating,2)
        matchStat_row.loc[:,'away_defend_rating'] = round(away_defend_rating,2)
        
        df_MergeMatchStatsSoFifa = pd.concat([df_MergeMatchStatsSoFifa,matchStat_row],ignore_index=True)

    df_MergeMatchStatsSoFifa.to_csv('./data/MergeMatchStatsSoFifa.csv',index=False,encoding='utf-8-sig')

MergeMatchStatsSoFifa(MergedSoFifaFbref,fbref_MergedMatchStats)

C:\Users\super\AppData\Local\Temp\ipykernel_4240\702448448.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_MergeMatchStatsSoFifa = pd.concat([df_MergeMatchStatsSoFifa,matchStat_row],ignore_index=True)
